In [ ]:
import numpy as np
from sklearn.model_selection import KFold # For K-fold cross-validation
from sklearn.preprocessing import LabelBinarizer # For one-hot encoding

# --- 1. Helper Functions for Activation and Loss ---

def relu(Z):
    """ReLU activation function."""
    return np.maximum(0, Z) # [1]

def relu_prime(Z):
    """Derivative of ReLU activation function."""
    return (Z > 0).astype(float) # [1]

def softmax(Z):
    """Softmax activation function for the output layer."""
    exp_Z = np.exp(Z - np.max(Z, axis=1, keepdims=True)) # Subtract max for numerical stability
    return exp_Z / np.sum(exp_Z, axis=1, keepdims=True) # [1]

def cross_entropy_loss(Y, Y_hat):
    """Multiclass cross-entropy loss function."""
    m = Y.shape[0]  # Fixed: get number of samples correctly
    # Avoid log(0) by clipping predictions
    Y_hat_clipped = np.clip(Y_hat, 1e-12, 1 - 1e-12)
    loss = -np.sum(Y * np.log(Y_hat_clipped)) / m # [1]
    return loss

def calculate_accuracy(Y, Y_hat):
    """Calculates classification accuracy."""
    predictions = np.argmax(Y_hat, axis=1)
    true_labels = np.argmax(Y, axis=1)
    accuracy = np.mean(predictions == true_labels) # [1]
    return accuracy

# --- 2. Adam Optimizer ---

class AdamOptimizer:
    """
    Adam (Adaptive Moment Estimation) optimizer.
    Utilizes adaptive moments for improved convergence. [1]
    """
    def __init__(self, learning_rate=0.01, beta1=0.9, beta2=0.999, epsilon=1e-8):
        self.learning_rate = learning_rate # [1]
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.m = {} # First moment estimates
        self.v = {} # Second moment estimates
        self.t = 0  # Time step

    def update_parameters(self, parameters, gradients):
        self.t += 1
        for key in parameters:
            if key not in self.m:
                self.m[key] = np.zeros_like(parameters[key])
                self.v[key] = np.zeros_like(parameters[key])

            # Update biased first and second moment estimates
            self.m[key] = self.beta1 * self.m[key] + (1 - self.beta1) * gradients[key]
            self.v[key] = self.beta2 * self.v[key] + (1 - self.beta2) * (gradients[key] ** 2)

            # Compute bias-corrected first and second moment estimates
            m_hat = self.m[key] / (1 - self.beta1 ** self.t)
            v_hat = self.v[key] / (1 - self.beta2 ** self.t)

            # Update parameters
            parameters[key] -= self.learning_rate * m_hat / (np.sqrt(v_hat) + self.epsilon)

# --- 3. Neural Network (Multilayer Perceptron) ---

class NeuralNetwork:
    """
    Multilayer Perceptron (MLP) for multiclass classification.
    Adapted from binary classification to handle 33 distinct classes. [1]
    """
    def __init__(self, input_size, hidden_layers_neurons, output_size, learning_rate=0.01, lambda_reg=0.01):
        self.input_size = input_size
        self.hidden_layers_neurons = hidden_layers_neurons # Configurable [1]
        self.output_size = output_size
        self.learning_rate = learning_rate # Initial for Adam [1]
        self.lambda_reg = lambda_reg # L2 Regularization strength [1]
        self.parameters = {}
        self.cache = {} # To store intermediate values for backpropagation

        self._initialize_parameters()

    def _initialize_parameters(self):
        """Initializes weights and biases for all layers."""
        layer_dims = [self.input_size] + self.hidden_layers_neurons + [self.output_size]
        for l in range(1, len(layer_dims)):
            # He initialization for ReLU
            self.parameters[f'W{l}'] = np.random.randn(layer_dims[l-1], layer_dims[l]) * np.sqrt(2. / layer_dims[l-1])
            self.parameters[f'b{l}'] = np.zeros((1, layer_dims[l]))

    def forward(self, X):
        """
        Performs forward propagation through the network. [1]
        X: Input data (m, input_size)
        """
        A = X
        self.cache['A0'] = X # Store input for backprop

        # Hidden layers with ReLU activation
        for l in range(1, len(self.hidden_layers_neurons) + 1):
            W = self.parameters[f'W{l}']
            b = self.parameters[f'b{l}']
            Z = np.dot(A, W) + b # Z[l] = A[l-1]W[l] + b[l][1]
            A = relu(Z) # A[l] = g[l](Z[l]) [1]
            self.cache[f'Z{l}'] = Z
            self.cache[f'A{l}'] = A

        # Output layer with Softmax activation
        W_out = self.parameters[f'W{len(self.hidden_layers_neurons) + 1}']
        b_out = self.parameters[f'b{len(self.hidden_layers_neurons) + 1}']
        Z_out = np.dot(A, W_out) + b_out # Z = A[1]W + b[1]
        A_out = softmax(Z_out) # A = softmax(Z) [1]
        self.cache[f'Z{len(self.hidden_layers_neurons) + 1}'] = Z_out
        self.cache[f'A{len(self.hidden_layers_neurons) + 1}'] = A_out # Y_hat

        return A_out

    def backward(self, X, Y, Y_hat):
        """
        Performs backpropagation to compute gradients. [1]
        X: Input data
        Y: True labels (one-hot encoded)
        Y_hat: Predicted probabilities from forward pass
        """
        m = X.shape[0]  # Fixed: get number of samples correctly
        gradients = {}
        num_layers = len(self.hidden_layers_neurons) + 1 # Total layers (hidden + output)

        # Gradient for output layer (L=3)
        dZ = Y_hat - Y # dJ/dZ = Y_hat - Y [1]
        dW = np.dot(self.cache[f'A{num_layers-1}'].T, dZ) / m # dJ/dW[1]
        db = np.sum(dZ, axis=0, keepdims=True) / m # dJ/db[1]

        # Add L2 regularization term to weight gradients [1]
        dW += (self.lambda_reg / m) * self.parameters[f'W{num_layers}']

        gradients[f'dW{num_layers}'] = dW
        gradients[f'db{num_layers}'] = db

        # Gradients for hidden layers (L=2, 1)
        for l in reversed(range(1, num_layers)):
            dZ_prev = np.dot(dZ, self.parameters[f'W{l+1}'].T) # (dJ/dZ[l+1] * W[l+1].T) [1]
            dZ = dZ_prev * relu_prime(self.cache[f'Z{l}']) # dJ/dZ[l] =... * ReLU'(Z[l]) [1]

            dW = np.dot(self.cache[f'A{l-1}'].T, dZ) / m # dJ/dW[l][1]
            db = np.sum(dZ, axis=0, keepdims=True) / m # dJ/db[l][1]

            # Add L2 regularization term to weight gradients [1]
            dW += (self.lambda_reg / m) * self.parameters[f'W{l}']

            gradients[f'dW{l}'] = dW
            gradients[f'db{l}'] = db

        return gradients

    def update_parameters(self, gradients, optimizer):
        """
        Updates network parameters using the Adam optimizer. [1]
        """
        # Prepare parameters and gradients in a format suitable for AdamOptimizer
        params_to_update = {}
        grads_for_optimizer = {}
        
        # Add weights and biases to the parameters dictionary
        for l in range(1, len(self.hidden_layers_neurons) + 2):
            params_to_update[f'W{l}'] = self.parameters[f'W{l}']
            params_to_update[f'b{l}'] = self.parameters[f'b{l}']
            grads_for_optimizer[f'W{l}'] = gradients[f'dW{l}']
            grads_for_optimizer[f'b{l}'] = gradients[f'db{l}']

        optimizer.update_parameters(params_to_update, grads_for_optimizer)

        # Copy updated parameters back to self.parameters
        for key in params_to_update:
            self.parameters[key] = params_to_update[key]

    def train(self, X_train, Y_train, X_val=None, Y_val=None, epochs=100, batch_size=32):
        """
        Trains the neural network.
        X_train: Training features
        Y_train: Training labels (one-hot encoded)
        X_val: Validation features (optional)
        Y_val: Validation labels (optional)
        epochs: Number of training epochs
        batch_size: Size of mini-batches
        """
        m = X_train.shape[0]  # Fixed: get number of samples correctly
        optimizer = AdamOptimizer(learning_rate=self.learning_rate) # [1]

        for epoch in range(1, epochs + 1):
            # Shuffle data for each epoch
            permutation = np.random.permutation(m)
            shuffled_X = X_train[permutation, :]
            shuffled_Y = Y_train[permutation, :]

            epoch_loss = 0
            epoch_accuracy = 0
            num_batches = 0

            for i in range(0, m, batch_size):
                X_batch = shuffled_X[i:i + batch_size, :]
                Y_batch = shuffled_Y[i:i + batch_size, :]

                # Forward pass
                Y_hat = self.forward(X_batch)

                # Calculate loss
                loss = cross_entropy_loss(Y_batch, Y_hat)
                epoch_loss += loss

                # Backpropagation
                gradients = self.backward(X_batch, Y_batch, Y_hat)

                # Update parameters
                self.update_parameters(gradients, optimizer)

                # Calculate accuracy for batch
                batch_accuracy = calculate_accuracy(Y_batch, Y_hat)
                epoch_accuracy += batch_accuracy
                num_batches += 1

            avg_epoch_loss = epoch_loss / num_batches
            avg_epoch_accuracy = epoch_accuracy / num_batches

            print(f"Epoch {epoch}/{epochs}: Train Loss = {avg_epoch_loss:.4f}, Train Accuracy = {avg_epoch_accuracy:.4f}")

            if X_val is not None and Y_val is not None:
                val_Y_hat = self.forward(X_val)
                val_loss = cross_entropy_loss(Y_val, val_Y_hat)
                val_accuracy = calculate_accuracy(Y_val, val_Y_hat)
                print(f"  Validation Loss = {val_loss:.4f}, Validation Accuracy = {val_accuracy:.4f}")

# --- 4. Data Handling (Simulated/Placeholder) ---

def load_and_preprocess_amhcd_data(num_samples=25740, num_classes=33, img_size=32):
    """
    Placeholder function to simulate loading and preprocessing AMHCD data.
    In a real scenario, this would load actual images, resize, convert to grayscale,
    flatten, and normalize. [1]

    Returns:
        X (np.array): Simulated image data (num_samples, 1024 features)
        y (np.array): Simulated labels (num_samples,)
    """
    print("Simulating AMHCD data loading and preprocessing...")
    # AMHCD: 25,740 images, 33 characters, 780 images per character, 60 transcribers. [1]
    # Images resized to 32x32 pixels and flattened to 1024 features (implying grayscale). [1]

    # Simulate 1024 features (32*32 for grayscale)
    X = np.random.rand(num_samples, img_size * img_size)
    # Normalize pixel values to [0, 1]
    X = X / np.max(X)

    # Simulate labels (0 to 32 for 33 classes)
    y = np.random.randint(0, num_classes, num_samples)

    print(f"Simulated data shape: X={X.shape}, y={y.shape}")
    return X, y

def augment_data(X, y, max_rotation_angle=15, max_translation_pixels=2):
    """
    Placeholder for data augmentation (rotations and translations). [1]
    In a real implementation, this would apply transformations to images.
    For this conceptual code, we'll just duplicate and add noise to simulate augmentation.
    """
    print(f"Applying data augmentation (simulated)...")
    # Simulate adding rotated/translated versions by adding noise
    X_augmented = np.vstack([X, X + np.random.normal(0, 0.05, X.shape)])
    y_augmented = np.hstack([y, y])
    print(f"Augmented data shape: X_augmented={X_augmented.shape}, y_augmented={y_augmented.shape}")
    return X_augmented, y_augmented

# --- 5. Main Execution with K-fold Cross-Validation ---

if __name__ == "__main__":
    # Hyperparameters [1]
    INPUT_SIZE = 1024 # 32x32 grayscale image flattened
    # Hidden layers are configurable, starting with 64 and 32 neurons. [1]
    HIDDEN_LAYERS_NEURONS = [64, 32]  # Fixed: added missing values
    OUTPUT_SIZE = 33 # 33 TifinaghIRCAM characters [1]
    LEARNING_RATE = 0.01 # [1]
    LAMBDA_REG = 0.001 # L2 regularization strength [1]
    EPOCHS = 50 # Example value, should be tuned [1]
    BATCH_SIZE = 64 # Example value, should be tuned [1]
    K_FOLDS = 5 # K for K-fold cross-validation [1]

    # Load and preprocess simulated AMHCD data
    X_raw, y_raw = load_and_preprocess_amhcd_data()

    # Apply data augmentation [1]
    X_augmented, y_augmented = augment_data(X_raw, y_raw)

    # One-hot encode labels
    label_binarizer = LabelBinarizer()
    Y_one_hot = label_binarizer.fit_transform(y_augmented)

    # K-fold Cross-Validation [1]
    kf = KFold(n_splits=K_FOLDS, shuffle=True, random_state=42)
    fold_accuracies = []  # Fixed: initialized empty list
    fold_losses = []      # Fixed: initialized empty list

    print(f"\nStarting {K_FOLDS}-fold cross-validation...")

    for fold, (train_index, val_index) in enumerate(kf.split(X_augmented)):
        print(f"\n--- Fold {fold + 1}/{K_FOLDS} ---")
        X_train, X_val = X_augmented[train_index], X_augmented[val_index]
        Y_train, Y_val = Y_one_hot[train_index], Y_one_hot[val_index]

        # Initialize a new model for each fold
        model = NeuralNetwork(
            input_size=INPUT_SIZE,
            hidden_layers_neurons=HIDDEN_LAYERS_NEURONS,
            output_size=OUTPUT_SIZE,
            learning_rate=LEARNING_RATE,
            lambda_reg=LAMBDA_REG
        )

        # Train the model for the current fold
        model.train(X_train, Y_train, X_val, Y_val, epochs=EPOCHS, batch_size=BATCH_SIZE)

        # Evaluate final performance on validation set for this fold
        final_val_Y_hat = model.forward(X_val)
        final_val_loss = cross_entropy_loss(Y_val, final_val_Y_hat)
        final_val_accuracy = calculate_accuracy(Y_val, final_val_Y_hat)

        print(f"Fold {fold + 1} Final Validation Loss: {final_val_loss:.4f}")
        print(f"Fold {fold + 1} Final Validation Accuracy: {final_val_accuracy:.4f}")

        fold_losses.append(final_val_loss)
        fold_accuracies.append(final_val_accuracy)

    # Report average K-fold results
    avg_accuracy = np.mean(fold_accuracies)
    std_accuracy = np.std(fold_accuracies)
    avg_loss = np.mean(fold_losses)

    print("\n--- K-fold Cross-Validation Results ---")
    print(f"Average Validation Accuracy: {avg_accuracy:.4f} +/- {std_accuracy:.4f}")
    print(f"Average Validation Loss: {avg_loss:.4f}")
    print("---------------------------------------")

    print("\nThis code provides a conceptual implementation based on the report's requirements.")
    print("For a full, runnable solution, you would need to:")
    print("1. Download the AMHCD dataset from the specified Kaggle link.[1]")
    print("2. Implement actual image loading, resizing, grayscale conversion, and flattening.")
    print("3. Implement robust data augmentation (e.g., using scikit-image or OpenCV).")
    print("4. Potentially use a deep learning framework (TensorFlow/PyTorch) for performance.")
    print("5. Conduct thorough hyperparameter tuning for hidden layer neurons, learning rate, lambda_reg, epochs, and batch size to achieve the 'best score'.[1]")
    print("6. Address computational resource difficulties, possibly by using GPUs if available.[1]")
    print("7. Provide a public GitHub repository link in the final report.[1]")

Simulating AMHCD data loading and preprocessing...
Simulated data shape: X=(25740, 1024), y=(25740,)
Applying data augmentation (simulated)...
Augmented data shape: X_augmented=(51480, 1024), y_augmented=(51480,)

Starting 5-fold cross-validation...

--- Fold 1/5 ---
Epoch 1/50: Train Loss = 3.5026, Train Accuracy = 0.0316
  Validation Loss = 3.4964, Validation Accuracy = 0.0324
Epoch 2/50: Train Loss = 3.4981, Train Accuracy = 0.0320
  Validation Loss = 3.4978, Validation Accuracy = 0.0286
Epoch 3/50: Train Loss = 3.4982, Train Accuracy = 0.0309
  Validation Loss = 3.4968, Validation Accuracy = 0.0341
Epoch 4/50: Train Loss = 3.4982, Train Accuracy = 0.0323
  Validation Loss = 3.4958, Validation Accuracy = 0.0334
Epoch 5/50: Train Loss = 3.4980, Train Accuracy = 0.0327
  Validation Loss = 3.4986, Validation Accuracy = 0.0302
Epoch 6/50: Train Loss = 3.4981, Train Accuracy = 0.0294
  Validation Loss = 3.4975, Validation Accuracy = 0.0287
Epoch 7/50: Train Loss = 3.4983, Train Accuracy 